# Part 1 V2
This script is meant to be an update to the original Part 1, simplifying the functions. Some changes to original part 1 code will be needed. 

In [1]:
import pandas as pd
import numpy as np
import os

# Download All of the Data

In [23]:
# Information
DATASOURCE_IDS = ['35', '38', '41', '42']
j_drive_output_folder_path = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/'
geography_levels = ['mgra_id', 'mgra', 'cpa', 'LUZ', 'jurisdiction', 'region']

In [4]:
# download all of the data 
all_data = {}
for ds_id in DATASOURCE_IDS:
    temp_dict = {}
    for file in os.listdir(j_drive_output_folder_path):
        if (ds_id in file):
            temp_dict[file] = pd.read_csv(j_drive_output_folder_path + file)
    all_data[ds_id] = temp_dict

# Both Files

In [8]:
def create_both_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"forecast_{DS1}_{level}_ind.csv"]
    df2 = all_data[DS2][f"forecast_{DS2}_{level}_ind.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print("Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")
    elif len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
    elif len(set(df1['yr_id']).difference(set(df2['yr_id']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['yr_id'])) if x not in list(set(df2['yr_id']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['yr_id'])) if x not in list(set(df1['yr_id']))]}")
    
    output = df1.merge(df2, on=[f'{level}','yr_id'], how='left', suffixes=[f'_DS{DS1}', f'_DS{DS2}'])
    output = output.groupby([f'{level}', 'yr_id']).sum()

    if send_to_j_drive == 'Y':
        output.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/' + f"{level}_both_DS{DS1}_DS{DS2}_QA.csv", index=True) 

    return output

In [24]:
both_file = create_both_file(DS1='35', DS2='41', level='jurisdiction', send_to_j_drive='N')

In [25]:
both_file

10 to 14_DS35  15 to 17_DS35  18 and 19_DS35  \
jurisdiction yr_id                                                 
Carlsbad     2016            7051           4610            2643   
             2018            6708           4284            2952   
             2020            6772           4163            3004   
             2025            7003           4162            2956   
             2030            6244           4260            3160   
...                           ...            ...             ...   
Vista        2030            7301           5044            3933   
             2035            7126           4325            3491   
             2040            6505           4239            3412   
             2045            6256           3907            3177   
             2050            6592           3872            3066   

                    20 to 24_DS35  25 to 29_DS35  30 to 34_DS35  \
jurisdiction yr_id                                                
Carlsbad     2016            7423           6550           6887   
             2018            8310           7793           7840   
             2020            8043           7914           8058   
             2025            7923           7205           7997   
             2030            8372           7308           7261   
...                           ...            ...            ...   
Vista        2030            9545           7710           7013   
             2035            9800           8817           7568   
             2040            8778           8875           8554   
             2045            8601           7668           8552   
             2050            7967           7726           7562   

                    35 to 39_DS35  40 to 44_DS35  45 to 49_DS35  5 to 9_DS35  \
jurisdiction yr_id                                                             
Carlsbad     2016            6991           7000           8018         6630   
             2018            7695           6738           7615         6872   
             2020            8084           7040           6837         6954   
             2025            7985           7917           6802         6185   
             2030            7572           7665           7721         6004   
...                           ...            ...            ...          ...   
Vista        2030            6819           6304           6500         6925   
             2035            6442           6677           5998         6353   
             2040            7014           6269           6250         6118   
             2045            7951           6805           5918         6435   
             2050            7979           7577           6564         6464   

                    ...  Retail Trade_DS41  Self-Employed_DS41  \
jurisdiction yr_id  ...                                          
Carlsbad     2016   ...               6799                   0   
             2018   ...               7337                 749   
             2020   ...               8134                 769   
             2025   ...               9396                 789   
             2030   ...              10389                 797   
...                 ...                ...                 ...   
Vista        2030   ...               5398                 507   
             2035   ...               5410                 525   
             2040   ...               5419                 535   
             2045   ...               5476                 536   
             2050   ...               5526                 547   

                    Transporation, Warehousing, and Utilities_DS41  \
jurisdiction yr_id                                                   
Carlsbad     2016                                             1538   
             2018                                             1545   
             2020                                             1545   
             

# Diff Files